In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jan_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jan_dict_JSON = json.load(json_file)

In [11]:
week_day_Jan_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]
#week_day_Jan_list = [2, 9, 16, 23, 30]

In [12]:
link_day_minute_Jan_list = []
for link_idx in range(24):
    for day in week_day_Jan_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jan_list.append(link_day_minute_Jan_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [13]:
len(link_day_minute_Jan_list)

63360

In [14]:
x = np.matrix(link_day_minute_Jan_list)
x = np.matrix.reshape(x, 24, 2640)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [15]:
np.size(x, 0), np.size(x, 1)

(24, 2640)

In [16]:
x[:,:2]

matrix([[  2549.89020931,   3473.34628721],
        [  2216.97886641,   2314.75710284],
        [  4683.72502323,   4473.83941121],
        [  4977.59590361,   5415.02072177],
        [  2678.23030098,   1528.36765517],
        [  1370.37941264,   2350.96391052],
        [  2751.3851227 ,   3564.13858436],
        [  3069.18157878,   2370.34401583],
        [  2538.07439233,   6373.4112037 ],
        [  5474.63715741,   4205.39275477],
        [  4976.33139471,   3723.16269501],
        [  8126.4315399 ,  10312.59241407],
        [  5726.49802046,   4685.52529624],
        [  2044.33945823,   4498.48015026],
        [  5825.38372421,   6698.98632309],
        [  2523.90644068,   3102.76257684],
        [  2833.52610072,   2173.77555997],
        [  3771.43907957,   2616.80352047],
        [  3944.44735314,   3382.27232185],
        [  3740.19651472,   2945.8934212 ],
        [  5013.82039391,   2843.93860165],
        [  2006.93802715,   2384.89770783],
        [  2817.36240904,   2523

In [17]:
np.size(A,0), np.size(A,1)

(24, 140)

In [18]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [3e-02, 2e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.01s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.55405287e+06  0.00000000e+00  4.88e+03 1.24e+02  9.54e+05     0s
   1   8.93658237e+05 -1.65160775e+06  7.81e+02 1.50e+02  1.93e+05     0s
   2   8.35496141e+05 -1.22861863e+06  6.25e+00 9.18e-01  3.80e+04     0s
   3   4.13130716e+04 -2.64279553e+05  6.25e-06 1.14e-13  5.4

In [19]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jan_weekday_AM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [20]:
lam_list

[3.6295062281518074e-11,
 4402.209708270412,
 3.9839950364271444e-11,
 9.419918289263449e-07,
 551.678221678437,
 1.5792667873841694e-06,
 1.1931947504304461e-08,
 0.0005241757138681469,
 1035.1047821530676,
 2720.980404765787,
 9.419916542215373e-07,
 386.14877787970346,
 1.5792662744402883e-06,
 1.588310240036771e-06,
 1750.8015827729675,
 1877.7444222120175,
 9.419824309869887e-07,
 9.419917337094333e-07,
 1113.7065158509017,
 10229.949554276745,
 1.5789305213030679e-06,
 1697.05225618085,
 1222.157180159464,
 1391.580844300555,
 1724.7765217635342,
 1479.2692326287827,
 1118.6354707698672,
 2599.7623853398622,
 0.1732306305327178,
 0.17266004657855616,
 5959.819336873942,
 1.0202175022228346,
 5493.811841620787,
 1885.8038026363536,
 756.4764696817406,
 3214.31098816818,
 2350.48374072759,
 7.860415910722091e-06,
 2127.4972041606056,
 69.74708599091525,
 4585.081808779965,
 862.2223611208549,
 2.5399569958678227e-07,
 2.5399569595971943e-07,
 6738.582890073613,
 1.2924656017773365e